In [1]:
import sys
sys.path.insert(0, '..')

import os
import requests


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'
print(os.path.exists(model_filepath))

tumor_control_model = 'BIOMD0000000749'
requests.get('https://www.ebi.ac.uk/biomodels/BIOMD0000000749', headers={'accept': 'json'})

True


<Response [200]>

In [2]:
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder, BuildPrompter
from biosimulator_processes.data_model import TimeCourseModel, TimeCourseModelChanges, ModelChanges, ModelChange

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
DeterministicTimeCourseParameterScan not available. Error: cannot import name 'TimeCourseProcessConfig' from 'biosimulator_processes.data_model' (/Users/alex/Desktop/uchc_work/repos/biosimulator-processes/notebooks/../biosimulator_processes/data_model.py)
